In [52]:
data_dir = "/home/cadel/data/deduce/Bej04/synteny"

# Load UCEs from BED file per genome

In [53]:
from functools import reduce
def load_uces():
    beds = []
    genome_names = []
    for f in os.listdir(data_dir):
        if f.endswith(".bed"):
            path = os.path.join(data_dir, f)
            name = f.replace(".bed", "")
            genome_names.append(name)
            beds.append(pd.read_csv(path, sep='\t', index_col=3, names=[x + "_" + name for x in ["chromosome", "start", "end", "uce_id", "identity"]]))
    
    return reduce(lambda acc, new: acc.join(new), beds), genome_names
    
uces, genome_names = load_uces()
uces['id'] = uces.index

# Find all "syntenic blocks"

A syntenic block contains >=$n$ UCEs such that:

* the UCEs appear on the same contig in each genome
* the UCEs appear in the same order in each genome

In [54]:
import numpy as np

def get_syntenic_blocks(uces, genome_names, min_size=2):
    # Build the condition for runs
    cond = pd.Series(np.repeat(0, len(uces)), index=uces.index)
   
    for genome in genome_names:
        # Chromosome must be the same
        cond |= (uces[f"chromosome_{genome}"] != uces[f"chromosome_{genome}"].shift())
        # Position must be the ascending
        cond |= (uces[f"start_{genome}"] < uces[f"start_{genome}"].shift())
    
    agg = {"size": pd.NamedAgg(column=f"chromosome_{genome_names[0]}", aggfunc="size"),
          "unique_size": pd.NamedAgg(column="id", aggfunc=lambda xs: len(set(xs)))}
    
    for genome in genome_names:
        agg[f"start_{genome}"] = pd.NamedAgg(column=f"start_{genome}", aggfunc="min")
        agg[f"end_{genome}"] = pd.NamedAgg(column=f"end_{genome}", aggfunc="max")
        agg[f"chromosome_{genome}"] = pd.NamedAgg(column=f"chromosome_{genome}", aggfunc="first")
    
    agg["id"] = pd.NamedAgg(column="id", aggfunc=lambda xs: ','.join(map(str,xs)))

    runs = uces.groupby(cond.cumsum()).agg(**agg)
    return runs[runs["unique_size"] >= min_size]

blocks = get_syntenic_blocks(uces, genome_names, min_size=2)

,size,unique_size,start_bej04_200bp_100_minimap_7f14d17-mm3,end_bej04_200bp_100_minimap_7f14d17-mm3,chromosome_bej04_200bp_100_minimap_7f14d17-mm3,start_bej04_200bp_100_minimap_7f14d17-hg16,end_bej04_200bp_100_minimap_7f14d17-hg16,chromosome_bej04_200bp_100_minimap_7f14d17-hg16,start_bej04_200bp_100_minimap_7f14d17-rn3,end_bej04_200bp_100_minimap_7f14d17-rn3,chromosome_bej04_200bp_100_minimap_7f14d17-rn3,id
34,2,2,58428541,62476164,chr2,157862654,162297897,chr2,39248658,43374319,chr3,"33,34"
111,2,2,45388244,62593657,chr2,144973081,162441529,chr2,24998191,43582570,chr3,"111,112"
168,2,2,44274496,45145030,chr19,102039686,102931885,chr10,249939121,250914602,chr1,"169,170"
171,2,2,109508047,111482390,chr14,93316882,95707071,chr13,103353631,105495816,chr15,"173,174"
279,2,2,126029763,126240915,chr7,130920737,131166306,chr10,196934579,197133087,chr1,"282,283"
363,2,2,62624347,65212446,chr2,162475570,165046913,chr2,43551807,46213312,chr3,"360,361"
443,2,2,78124039,95260853,chrX,24370511,80545710,chrX,80729034,98554498,chrX,"441,442"
459,2,2,59418052,100148285,chr12,43569060,95421660,chr14,86407805,130308089,chr6,"458,459"
465,2,2,14968003,16395019,chr6,113617521,114874201,chr7,40840917,42213059,chr4,"465,466"


In [55]:
# Map UCEs in syntenic blocks to positions in the assembly
# Read positions in per UCE
# Check each syntenic block and count the ones that are correct / incorrect in the assembly


In [154]:
from collections import OrderedDict
bos_index = "../uce_def/Mus_musculus.GRCm39.dna.toplevel.fa.index"
def get_chromosome_metadata(fa_index):
    with open(fa_index, "r") as f:
        reader = csv.reader(f, delimiter=' ')
        chromosome_positions = sorted([(line[0], int(line[1])) for line in reader], key=lambda x: int(x[1]))

    chromosome_lengths = {}
    for i in range(len(chromosome_positions) - 1):
        chromosome_lengths[chromosome_positions[i][0]] = chromosome_positions[i+1][1] - chromosome_positions[i][1]
    return chromosome_lengths

def get_chromosome_metadata_no_contigs(fa_index):
    return OrderedDict([(k, v) for k, v in get_chromosome_metadata(fa_index).items() if is_int(k) or k in ["X", "Y"]])
    

In [147]:
import matplotlib.pyplot as plt

colour_map = plt.cm.get_cmap('hsv', len(runs))
pd.DataFrame

(1.0, 0.7875007875007873, 0.0, 1.0)

In [155]:
from reportlab.lib.units import cm
from reportlab.lib import colors

from Bio import SeqIO
from Bio.Graphics import BasicChromosome

genome_name = "very_relaxed_uces-mus_musculus"
genome_chromosomes = get_chromosome_metadata_no_contigs(bos_index)
d = BasicChromosome.Organism()

#http://biopython.org/DIST/docs/tutorial/Tutorial.html#sec355
d.page_size = (29.7 * cm, 21 * cm)  # A4 landscape

features = []

for i, f in enumerate(runs.filter(items=[f"start_{genome_name}", f"end_{genome_name}", f"chromosome_{genome_name}"]).itertuples(name=None)):
    colour = colour_map(i)
    features.append((f[3], (f[1], f[2], None, str(f[0]), colors.Color(*colour))))

for chromosome_name, chromosome_length in genome_chromosomes.items():
    d_chromosome = BasicChromosome.Chromosome(chromosome_name)
    
    chromosome_features = [f[1] for f in features if f[0] == chromosome_name]
    print(chromosome_features)
    body = BasicChromosome.AnnotatedChromosomeSegment(chromosome_length, chromosome_features)
    body.scale = chromosome_length
    d_chromosome.add(body)
    d.add(d_chromosome)

d.draw("mus.pdf", "Mus musculus")



[(59355516.0, 59355701.0, None, '85285', Color(0,.456252,1,1))]
[(114243014.0, 114243294.0, None, '157621', Color(1,0,.881251,1))]
[(18768037.0, 18768477.0, None, '17395', Color(1,0,0,1))]
[(44852090.0, 44852275.0, None, '62307', Color(.818749,1,0,1))]
[]
[]
[(136811305.0, 136813348.0, None, '179621', Color(1,0,.09375,1))]
[]
[]
[]
[]
[(46744806.0, 46745499.0, None, '66237', Color(.031249,1,.000001,1)), (50167202.0, 50167564.0, None, '70973', Color(0,1,.3625,1)), (53880215.0, 53880395.0, None, '77772', Color(0,1,.756248,1))]
[]
[(97775220.0, 97775485.0, None, '136844', Color(.331248,0,1,1)), (98773879.0, 98774314.0, None, '139259', Color(.724999,0,1,1)), (120641940.0, 120642179.0, None, '168496', Color(1,0,.4875,1))]
[]
[]
[(86337643.0, 86337862.0, None, '121553', Color(0,.062502,1,1))]
[(36421569.0, 36421837.0, None, '45682', Color(1,.39375,0,1)), (42685581.0, 42685785.0, None, '57499', Color(1,.787501,0,1))]
[(45435215.0, 45435563.0, None, '63934', Color(.424998,1,0,1)), (55465466.0,

In [80]:
import matplotlib
g = sns.relplot(data=blocks, x="very_relaxed_uces-homo_sapiens_pos", y="very_relaxed_uces-mus_musculus_pos", col="very_relaxed_uces-homo_sapiens_chr", row="very_relaxed_uces-mus_musculus_chr", facet_kws={"margin_titles":True,"despine":False}, row_order=list(range(19,0,-1)))
g.set(xticklabels=[])
g.set(xlabel=None)
g.set(yticklabels=[])
g.set(ylabel=None)
g.tight_layout()
g.fig.subplots_adjust(wspace=0, hspace=0)
g.set_titles(col_template="chr_{col_name}", row_template="chr_{row_name}")

g.savefig("facet_plot.pdf")
#matplotlib.pyplot.show()
#g

In [1]:
from reportlab.lib.units import cm
from Bio import SeqIO
from Bio.Graphics import BasicChromosome


In [73]:
sns.FacetGrid(data=blocks, col="very_relaxed_uces-homo_sapiens_chr", row="very_relaxed_uces-mus_musculus_chr")

Error in callback <function flush_figures at 0x0000017427967D30> (for post_execute):


KeyboardInterrupt: 

In [41]:
import itertools



    
chromosomes_by_genome = {}
for genome_name in genomes:
    chromosomes = set(int(x.chromosome) for x in bed_files[genome_name] if is_int(x.chromosome))
    chromosomes_by_genome[genome_name] = chromosomes

chromosome_product = list(itertools.product(*chromosomes_by_genome.values()))
chromosome_product[1]

(1, 1, 1, 1, 2)